# 2. Génération de données synthétiques téléphonie + GenAI (RAG) + sécurité

Ce notebook guide pas à pas la création de données simulées pour un projet téléphonie, avec un focus sur la sécurité et l'utilisation future en GenAI/RAG.

## Objectif du notebook

- Générer un fichier CSV d'appels téléphoniques (CDR) avec masquage des données personnelles.
- Créer des fichiers de dialogues synthétiques pour chaque thème métier.
- Expliquer chaque étape de façon pédagogique.

## Étapes du processus

1. Importer les librairies
2. Charger les variables d'environnement
3. Définir les paramètres
4. Masquer la PII
5. Générer les appels
6. Analyser le CSV
7. Définir les templates de dialogues
8. Générer les transcripts
9. Conclusion

### Étape 1 : Import des librairies
Dans cette étape, on importe toutes les librairies nécessaires pour le projet. On utilise des librairies standards de Python (comme os, datetime, hashlib) et des librairies spécialisées pour la data science (pandas, numpy). La librairie dotenv permet de charger des variables d'environnement depuis un fichier .env, ce qui est utile pour la sécurité et la configuration.

In [ ]:
# ----- Import libraries PEP 8 -----
# ----- Standard library -----
import os 
import pandas as pd 
import numpy as np 
import hashlib 
from datetime import datetime, timedelta 
# ----- Third party libraries -----
from dotenv import load_dotenv # to use .env file

### Étape 2 : Chargement des variables d'environnement (.env)
Cette étape consiste à charger les variables d'environnement stockées dans le fichier `.env` du projet. On utilise la librairie `dotenv` pour cela. Le but est de récupérer le `SALT_PII`, une chaîne aléatoire qui sert à renforcer la sécurité lors du masquage des données personnelles (PII). Ce salt sera utilisé pour hacher les numéros de téléphone et garantir leur confidentialité dans les données générées.

In [ ]:
load_dotenv() # load .env file
# PII (Personally Identifiable Information) to mask sensitive data (name, firstname, phone number, email, etc.)
salt_pii = os.getenv("SALT_PII", "changeme") 

### Étape 3 : Création des dossiers de sortie
Ici, on prépare les dossiers où seront enregistrés les fichiers générés par le script. Le dossier `data/raw` contiendra les données brutes (CSV des appels), et le sous-dossier `transcripts` contiendra les fichiers de dialogues. On utilise la fonction `os.makedirs` pour créer ces dossiers si besoin, sans écraser ceux qui existent déjà.

In [ ]:
raw_directory = "../data/raw" # enregistrer les fichiers de données brutes (raw data)
transcripts_directory = os.path.join(raw_directory, "transcripts") # combine chemin de fichiers 
os.makedirs(raw_directory, exist_ok=True) # crée le dossier raw
os.makedirs(transcripts_directory, exist_ok=True) # crée le dossier transcripts s'il n'existe pas déjà.

### Étape 4 : Définition des paramètres de génération
Dans cette étape, on définit les paramètres qui contrôlent la génération des données : le nombre d'appels à simuler, le nombre de fichiers de dialogues à créer, la liste des agents fictifs et les différents thèmes d'appels. Ces paramètres permettent d'adapter le jeu de données à la taille et à la diversité souhaitées.

In [ ]:
numbers_of_calls = 200 # Nombre d'appels à générer dans le CSV
numbers_of_transcripts = 18 # Nombre de fichiers de dialogues à générer (pour couvrir tous les thèmes)
agents = ["A", "B", "C", "D", "E"] # Liste des identifiants d'agents (5 agents fictifs pour simuler la réalité)
topics = ["billing", "tech_support", "orders", "returns", "other"] # Liste des thèmes d'appels (facturation, support, commandes, retours, autres)

### Étape 5 : Fonction de masquage des données personnelles (PII)
Pour garantir la confidentialité des données, on utilise une fonction qui masque les numéros de téléphone en les transformant en une suite de caractères incompréhensible (hash). On utilise le salt récupéré précédemment pour renforcer la sécurité du hachage. Cette étape est essentielle pour respecter les bonnes pratiques de sécurité et d'anonymisation.

In [ ]:
def mask_pii(sensitive_data: str, salt: str = salt_pii) -> str: 
    """
    Hash une données sensible avec un salt pour masquer la PII.
    On utilise la boîte à outils hashlib pour : Prendre le sel et le numéro, 
    les coller ensemble, Les transformer en une suite de caractères incompréhensible (le hash), 
    Retourner ce hash comme résultat
    
    Args:
        sensitive_data (str): La donnée sensible à masquer (ex: numéro de téléphone).
        salt (str): Le salt cryptographique pour renforcer la sécurité.
    Returns:
        str: Le hash SHA-256 de la donnée sensible combinée avec le salt.
    """
    return hashlib.sha256((salt + sensitive_data).encode()).hexdigest() 

### Étape 6 : Génération des appels téléphoniques (CDR)
On génère ici les données simulant les appels téléphoniques : chaque appel a un identifiant, un numéro masqué, un agent, une date, une durée, un sujet et une résolution. On utilise des tirages aléatoires pour rendre le jeu de données réaliste et varié. Toutes les informations sont stockées dans une liste de dictionnaires, prête à être convertie en tableau de données.

In [ ]:
np.random.seed(42) # Fixation de la graine aléatoire pour que les résultats soient reproductibles (toujours les mêmes données générées)
start_date = datetime.now() - timedelta(days=30) # On choisit une date de départ : il y a 30 jours
cdr_rows = [] # Préparation d'une liste vide pour stocker chaque appel généré
for i in range(numbers_of_calls): # boucle qui répète l'opération 200 fois (numbers_of_calls), pour créer chaque appel
    call_id = f"CALL_{i+1:04d}" # Création d'un identifiant unique pour chaque appel, sous la forme CALL_0001, CALL_0002, etc.
    caller_number = f"04{np.random.randint(2000000,3999999)}" # On génère un numéro de téléphone fictif de l'appelant, qui commence par 04 et se termine par 7 chiffres aléatoires
    caller_id = mask_pii(caller_number) # Variable qui va contenir le numéro masqué (hashé) de l'appelant, en utilisant la fonction mask_pii définie plus haut
    agent_id = np.random.choice(agents) # Variable qui va contenir l'identifiant d'un agent choisi au hasard parmi la liste agents (.np.random.choice())

     # Variable qui va contenir une date et heure de début d'appel, répartie sur les 30 derniers jours (43200 minutes = 30 jours) : np.random.randint(0, 43200) 
     # tire au hasard un nombre de minutes entre 0 et 43200 (soit 30 jours × 24h × 60min). 
     # timedelta(minutes=...) ajoute ce nombre de minutes à la date de départ pour obtenir une date et heure répartie sur le dernier mois.
    start_timestamp = start_date + timedelta(minutes=np.random.randint(0, 43200)) 
    
    call_duration_seconds = np.random.randint(30, 1800) # Variable qui va contenir une durée d'appel aléatoire entre 30 secondes et 1800 secondes (soit 30 minutes max)
    topic = np.random.choice(topics) # Variable qui va contenir un sujet d'appel choisi au hasard parmi la liste topics
    resolved = int((call_duration_seconds < 900 and topic != "tech_support") or np.random.rand() > 0.3) # Décide si l'appel est résolu (1) ou non (0), en fonction de la durée, du sujet, et d'un peu de hasard
    cdr_rows.append({
        "call_id": call_id,
        "caller_id": caller_id,
        "agent_id": agent_id,
        "start_ts": start_timestamp.isoformat(),
        "duration_sec": call_duration_seconds,
        "topic": topic,
        "resolved": resolved
    })

### Étape 7 : Sauvegarde des appels dans un fichier CSV
Une fois les appels générés, on les convertit en DataFrame (tableau de données) avec pandas, puis on les sauvegarde dans un fichier CSV. Ce fichier pourra être utilisé pour l’analyse, l’entraînement de modèles ou l’intégration dans un système RAG.

In [27]:
cdr_df = pd.DataFrame(cdr_rows) # Variable qui va contenir un tableau de données (DataFrame) créé à partir de la liste cdr_rows. Chaque dictionnaire dans cdr_rows devient une ligne dans le tableau, avec les clés comme colonnes.
cdr_df.to_csv(os.path.join(raw_directory, "cdr_synthetic.csv"), index=False) # Export et enregistrement(.to_csv()) du tableau cdr_df dans un fichier CSV nommé cdr_synthetic.csv dans le dossier raw_directory. index=False signifie qu'on ne veut pas inclure l'index des lignes dans le fichier CSV.
print(f"Fichier CDR généré: {os.path.join(raw_directory, 'cdr_synthetic.csv')}") # Affiche un message pour indiquer où le fichier CSV a été enregistré

Fichier CDR généré: ../data/raw/cdr_synthetic.csv


## Analyse rapide du CSV généré

On vérifie le contenu du fichier généré et on affiche les premières lignes pour valider la structure.

### Étape 8 : Analyse rapide du CSV généré
On vérifie ici que le fichier CSV a bien été créé et que sa structure correspond à nos attentes. On affiche les premières lignes du tableau pour valider la génération des données.

In [28]:
cdr_df.head() # Affiche les 5 premières lignes du DataFrame pour vérifier la génération

,call_id,caller_id,agent_id,start_ts,duration_sec,topic,resolved
0,CALL_0001,b2b79fe6a94f1d82fa5972eaa6647146fd97d3cec89b53...,D,2025-08-16T07:42:51.470748,1324,orders,1
1,CALL_0002,9a414725d4d31b8e443e0f5f4ea3d9eb1c74f6291422f8...,B,2025-08-27T10:12:51.470748,1268,orders,1
2,CALL_0003,aab3e738cb3d19170392f1516db9aae2398b40af97a0f9...,D,2025-08-26T20:25:51.470748,160,other,1
3,CALL_0004,34f186ba3c2f4059776724f32f746e17a124f51cbba67f...,D,2025-08-17T09:55:51.470748,1245,returns,0
4,CALL_0005,ea24093ef6253cec250cd830d4a8ae0692f362c539bf23...,B,2025-09-04T15:35:51.470748,282,returns,1


### Étape 9 : Définition des templates de dialogues
Pour simuler des conversations téléphoniques, on prépare des modèles de dialogues pour chaque thème métier (facturation, support, commandes, retours, autres). Chaque template contient plusieurs échanges typiques entre un client et un agent.

In [29]:
dialogue_templates = { # Dictionnaire qui contient des templates de dialogues pour chaque sujet. Chaque sujet (clé) a une liste de lignes de dialogue (valeur) entre un client et un agent.
    "billing": [
        "Client: Bonjour, j'ai une question sur ma facture.",
        "Agent: Bien sûr, pouvez-vous préciser votre demande ?",
        "Client: Il y a un montant que je ne comprends pas.",
        "Agent: Je vérifie cela pour vous."
    ],
    "tech_support": [
        "Client: Mon internet ne fonctionne plus.",
        "Agent: Avez-vous essayé de redémarrer votre box ?",
        "Client: Oui, mais ça ne marche toujours pas.",
        "Agent: Je vais lancer un diagnostic."
    ],
    "orders": [
        "Client: Je souhaite suivre ma commande.",
        "Agent: Pouvez-vous me donner votre numéro de commande ?",
        "Client: C'est le 12345.",
        "Agent: Elle est en cours de livraison."
    ],
    "returns": [
        "Client: Je veux retourner un produit.",
        "Agent: Quelle est la raison du retour ?",
        "Client: Il ne correspond pas à ma commande.",
        "Agent: Je lance la procédure de retour."
    ],
    "other": [
        "Client: J'ai une question générale.",
        "Agent: Je vous écoute.",
        "Client: Quels sont vos horaires d'ouverture ?",
        "Agent: Nous sommes ouverts de 8h à 18h."
    ]
}

### Étape 10 : Génération des fichiers de dialogues
On génère ici les fichiers de dialogues : pour chaque thème et chaque fichier, on personnalise le nom du client et on sauvegarde le dialogue dans un fichier texte. Cela permet d’obtenir des exemples variés et réalistes pour l’entraînement ou l’analyse.

In [ ]:
# Generation des fichiers de dialogues
for i in range(numbers_of_transcripts):
    topic = np.random.choice(topics) 
    dialogue = dialogue_templates[topic] 
    
    # Ajout d'une variation simple
    dialogue = [line.replace("Client", f"Client_{i+1}") for line in dialogue] # line.replace() remplace "Client" par "Client_1", "Client_2", etc. pour chaque ligne du dialogue, afin d'ajouter une variation simple et rendre chaque fichier unique.
    transcript_path = os.path.join(transcripts_directory, f"{topic}_{i+1:02d}.txt") # .path.join() crée le chemin complet vers le fichier de transcription en combinant le dossier transcripts avec un nom de fichier basé sur le sujet et un numéro séquentiel (par exemple, billing_01.txt, tech_support_02.txt, etc.)
    
    with open(transcript_path, "w", encoding="utf-8") as f: # open() ouvre le fichier en mode écriture ("w") avec l'encodage UTF-8 pour supporter les caractères spéciaux. as f crée un objet fichier (f) pour écrire dedans.
        f.write("\n".join(dialogue)) # .write() écrit le dialogue dans le fichier, en joignant les lignes avec des sauts de ligne ("\n".join(dialogue))
    print(f"Transcript généré: {transcript_path}") # Affiche un message pour indiquer où le fichier de transcription a été enregistré

print("Données synthétiques générées avec succès.") # Message final pour indiquer que tout s'est bien passé

Transcript généré: ../data/raw/transcripts/billing_01.txt
Transcript généré: ../data/raw/transcripts/other_02.txt
Transcript généré: ../data/raw/transcripts/other_03.txt
Transcript généré: ../data/raw/transcripts/tech_support_04.txt
Transcript généré: ../data/raw/transcripts/returns_05.txt
Transcript généré: ../data/raw/transcripts/billing_06.txt
Transcript généré: ../data/raw/transcripts/billing_07.txt
Transcript généré: ../data/raw/transcripts/orders_08.txt
Transcript généré: ../data/raw/transcripts/other_09.txt
Transcript généré: ../data/raw/transcripts/billing_10.txt
Transcript généré: ../data/raw/transcripts/other_11.txt
Transcript généré: ../data/raw/transcripts/billing_12.txt
Transcript généré: ../data/raw/transcripts/tech_support_13.txt
Transcript généré: ../data/raw/transcripts/returns_14.txt
Transcript généré: ../data/raw/transcripts/billing_15.txt
Transcript généré: ../data/raw/transcripts/other_16.txt
Transcript généré: ../data/raw/transcripts/tech_support_17.txt
Transcript

## Conclusion

Ce notebook a permis de générer des données synthétiques téléphonie, d'appliquer des bonnes pratiques de sécurité (masquage PII), et de créer des dialogues variés pour l'entraînement ou l'analyse GenAI/RAG.